# Reinforcement Learning - Lab

## Introduction
In this lab you will take what you have learned about Reinforcement Learning and build an AI capable of playing a game. That game is called CartPole and involves balancing a pole that is on a cart.
<br>
<br>
More info about the environment can be found here
<br>
http://gym.openai.com/envs/CartPole-v1/
<br>
<br>
This is a great first environment to try out RL on because it is siginificantly less expensive to train than other environments.

## Objectives
You will be able to:
* Build a DQN agent
* Train the agent to play CartPole, achieving an average score of at least 500

## Importing libraries
Let's start by importing the necessary libraries.

In [ ]:
import random
import numpy as np
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from collections import deque
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import gym
import pickle
import time

## Setting up environment
Now we need to create the environment. The maximum number of steps in the game is by default set to 200. For this lab we will want to achieve an average score of 500, so we need to increase the max episode steps to 1000.

In [ ]:
env = gym.make('CartPole-v1')
env._max_episode_steps = 1000

We can view the size of the action space by running the following command.

In [ ]:
env.action_space

This tells us that there are 2 possible actions - 0 and 1. In this environment 0 correspondings to the cart moving left and 1 corresponds to the cart moving right.

Below shows the observation space for CartPole. This is the data we will get for each state of the game. These 4 variables will be what we use to train our model. It is important to note that these values will be normalized to [0,1] before they are given to us in the state data.

    Observation: 
        Type: Box(4)
        Num	Observation                 Min         Max
        0	Cart Position             -4.8            4.8
        1	Cart Velocity             -Inf            Inf
        2	Pole Angle                 -24 deg        24 deg
        3	Pole Velocity At Tip      -Inf            Inf

Some addtional info can be seen below regarding how the game decides it has lost.

    Termination: 
        Pole Angle is more than 12 degrees
        Cart Position is more than 2.4 (center of the cart reaches the edge of the display)

Scoring in this game is simple - each step that is made without ending the game will grant +1 point. A step that terminates the game will grant -1 point.

## Baseline performance

Now let's get the environment up and running and have it play a couple games randomly so we can get an idea of the baseline performance. We will keep track of the score each game so that we can get an average per 100 games. We will also plot the scores of each game to get idea of the variation in scores.

In [ ]:
n_games = 100
n_steps = 1000
scores = []
for game in range(n_games):
    state = env.reset()
    for t in range(n_steps):
        env.render()
        action = env.action_space.sample()
        next_state, reward, done, info = env.step(action)
        if done:
            print('Game ' + str(game) + ' score: ' + str(t))
            scores.append(t)
            break
env.close()
plt.plot(scores)
plt.title('End Game Scores')
plt.xlabel('Game')
plt.ylabel('Score')

Our average score per 100 games is then:

In [ ]:
np.mean(scores)

## Building a DQN Agent
Now we can start building the agent that will learn to play CartPole.

We first want to choose and initialize our parameters.

For each state we are given 4 values, thus the input_shape for our network will be 4.

In [ ]:
input_shape = 4

The batch_size will be 64. This means that every time we perform experience replay, we will sample 64 different memories and train the model on those.

In [ ]:
batch_size = 64

We will set the total number of games to 2000. Almost certainly the model will reach its goal performance after less games (~500-1000) but just in case it takes longer we will extend the maximum number of games.
<br>
<br>
The number of steps will be set to 1000 which is the maxmimum number of steps we set initially.
<br>
<br>
We can also declare our score goal of 500 (avg per 100 games).

In [ ]:
n_games = 2000
n_steps = 1000
score_goal = 500

For our epsilon greedy policy, we will start epsilon off at 1 and decay it by 0.95 after each training session. This way, over time, the network will rely more and more on its own predictions for moves.

In [ ]:
epsilon = 1
epsilon_decay = 0.995

Gamma is our reward discount. This lessens the value of future rewards when calculating Q-values. The value is 0.95 is the most common value used.

In [ ]:
gamma = 0.95

Lastly, we want to initialize our memory. We will use an object known as a deque. A deque is a lot like a normal list, except that it has a max length and once that max length is reached, a new item added to the deque will cause the oldest item in the deque to be deleted. That way we only keep a maximum number of memories in storage at one time.

In [ ]:
memory = deque(maxlen=10000)

## Neural Network
Now let's build and intial both our primary model and target model.

This architecture was found mostly through trial and error. Feel free to modify it. Odds are there is a better architecture that will train the model faster.
<br>
<br>
Note that our input_dim is equal to our input_shape - 4. Also, the final layer of the network has 2 neurons - corresponding to our two possible actions: left or right. It is essential to use a linear activation function at the final layer.

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim=input_shape, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='linear'))
model.compile(loss="mse", optimizer=Adam(lr=0.001))
print(model.summary())

Now we can use the same architecture to create our target model.

In [ ]:
target_model = Sequential()
target_model.add(Dense(16, input_dim=input_shape, activation='relu'))
target_model.add(Dense(32, activation='relu'))
target_model.add(Dense(16, activation='relu'))
target_model.add(Dense(8, activation='relu'))
target_model.add(Dense(2, activation='linear'))
target_model.compile(loss="mse", optimizer=Adam(lr=0.001))

## Agent Methods
In order to train our models, we need to set up the functions required for a DQN.

The first two methods will help our agent choose an action. <br><br>
<b>choose_action</b> will generate a random number, if it is less than our current epsilon value, then a random move will be chosen. If the random number is greater than our current epsilon value, we will need to predict a value.

In [ ]:
def choose_action(state):
    if np.random.rand() <= epsilon:    
        action = random.choice([0,1])
    else:
        action = predict_action(state)
    return action

<b>predict_action</b> will take in a state and predict the Q-values for that state for each action. Then we can use argmax() to get the action with the highest Q-value and return that action.
<br>
<br>
Note that we are using the target model to predict here.

In [ ]:
def predict_action(state):
    q_values = target_model.predict(state)
    action = np.argmax(q_values[0])
    return action

Next we want to create a function that will store the agent's experiences into memory. The function is fairly straightforward, we will simply pass in the current state, the performed action, the reward, the next state, and the 'done' flag and append those values as a tuple into our memory deque.

In [ ]:
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

We need two additional helper functions - one to decay our epsilon value and one to transfer the weights of the primary model to the target model. These are both fairly straight forward.

In [ ]:
def decrease_epsilon():
    global epsilon
    epsilon *= epsilon_decay
    
def transfer_weights():
    target_model.set_weights(model.get_weights()) 

Now for probably the most complicated of the methods - experience replay.

The way this method works is, first we check and make sure we have enough experience in memory to create a batch of our selected batch_size.
<br>
<br>
Next we create our batches by randomly sampling our memory a number of times equal to batch_size.
<br>
<br>
Then we iterate through each batch. For each batch, we calculate the updated Q-value. The updated Q-value is calculated by using the Q-value formula. <br><br>
     <img src ='q_formula.png'></img>

Using this formula, the Q-value is calculated by adding together the current reward and the maximum possible Q-value of the next state, discount by our gamma value.
<br><br>
Note that we are using the target model to make this prediction.
<br><br>
Next we want to grab the current predicted Q-values for the given state.
<br>
<br>
We then update only the Q-value for the chosen action, leaving the other Q-value the same.
<br>
<br>
Our primary model is then fitted on the state and the Q-values.
<br>
<br>
Lastly, after each training session, we want to decay our epsilon value and also transfer the weights from the primary model to the target model.

In [ ]:
def experience_replay():
    if len(memory) < batch_size:
        return
    print('**training**')
    batch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in batch:
        q_update = reward
        if not done:
            q_update = (reward + gamma * np.amax(target_model.predict(next_state)[0]))
        q_values = target_model.predict(state)
        q_values[0][action] = q_update
        model.fit(state, q_values, verbose=0)
    decrease_epsilon() 
    transfer_weights()

In order to speed up training, we can adjust one of our input values - cart position - to force it into bins. This will help to shrink our overall state size. This works because the agent will still be learning the position of the cart on the rail, it will just be given more of a general position instead of a very specific position.
<br>
<br>
You don't have to worry too much about how exactly this code works, just know that each state needs to pass through the <b>cart_position_to_bin</b> function to adjust the cart position values.
<br>
<br>
Specifically we will be create 5 bins of cart positions. Feel free to experiment with more or less.

In [ ]:
n_bins = 5
bin_size = 1/n_bins
bins = []
for i in range(n_bins):
    bins.append((0, bin_size*i, i))

def cart_position_to_bin(state):
    position = state[0]
    for b in bins:
        if position>=b[0] and position<=b[1]:
            binned_position = b[2]
            state[0] = binned_position
            break
    return state

In order to get our state to be the correct size for the neural network, we also need to reshape the array. Below is a method to do this, coupled with the above cart position function to give you one state preprocessing method. Make sure to pass each state and next_state through this function.

In [ ]:
def preprocess_state(state):
    state = cart_position_to_bin(state)
    state = np.reshape(state, [1, input_shape])
    return state

All the prework is now complete. We have all the tools necessary to start training our agent!

## Training the agent
Alright! Now it's time to start training. To do this, we will need to build a loop that will play through games of CartPole, training our agent along the way.
<br><br>
Provided below is a basic skeleton of this loop. Use what you have learned from the lesson and this lab in order to complete it. If you get lost you can always check the solution.
<br><br>
(Warning: this process may take some time. It takes anywhere from 30-60 minutes on my machine but that may be more or less depending on your computer and GPU.)

In [ ]:
start_time = time.time()
scores = []
avgs = []
for game in range(n_games):
    ## Each game, we need to reset the environment, grab the intial state and preprocess it

    for t in range(n_steps):
        ## After each move, we need to do the following:
        ## 1) render the environment
        ## 2) decide on an action for this turn
        ## 3) perform that action, grabbing the next state, reward, 'done' flag, and 'info' object
        ## 4) preprocess the next state
        ## 5) remember this experience
        ## 6) set state=next_state to prepare for the next round
        ## 7) check if game is done. if so, record the score,
        ##    calculate the rolling average score (average score per last 100 games),
        ##    store the rolling average
        ## 8) check if average score goal has been reached, if so break out of game loop
        ## 9) if we need to keep playing, perform experience replay

        if done:
            print ("Game: " + str(game) + ", exploration: " + str(round(epsilon,2)) + ", score: " + str(t) + ' - avg score/100 games: ' + str(avg_100))
            break
    if avg_100 >= score_goal:
        print('Completed after ' + str(game) + ' games - Averaging over ' + str(score_goal) + 'points per 100 games')
        break
env.close()
print("--- %s seconds ---" % (time.time() - start_time))

Now that training is finished, let's plot the rolling average scores to see how the agent improved over time.

In [ ]:
plt.plot(avgs)
plt.title('Average score per 100 games')
plt.xlabel('Games (x100)')
plt.ylabel('Score')

## Summary
Congratulations! You just trained your first RL model! After completing the rite of passage that is CartPole. <br><br>
You are now eligible to try out other environments. 
A word of caution though - the Atari games may be the most exciting and enticing environments to train in, but they may take several days of training before convergence. They also may involve image data and the use of Convolutional Neural Networks so make sure you are comfortable with those first.

Finally, we can do a quick comparison of baseline CartPole vs our trained CartPole agent.

In [ ]:
n_games = 3
n_steps = 1000
scores = []
for game in range(n_games):
    state = env.reset()
    for t in range(n_steps):
        time.sleep(0.1)
        env.render()
        action = env.action_space.sample()
        next_state, reward, done, info = env.step(action)
        if done:
            print('Game ' + str(game) + ' score: ' + str(t))
            scores.append(t)
            break
env.close()

In [ ]:
n_games = 3
n_steps = 1000
scores = []
for game in range(n_games):
    state = env.reset()
    state = preprocess_state(state)
    for t in range(n_steps):
        time.sleep(0.1)
        env.render()
        action = predict_action(state)
        next_state, reward, done, info = env.step(action)
        if done:
            print('Game ' + str(game) + ' score: ' + str(t))
            scores.append(t)
            break
env.close()